In [146]:
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

csv_path = os.getenv('USZIPS_CSV_PATH')
df_zipcode = pd.read_csv(csv_path)
df_zipcode.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16721.0,100.2,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37510.0,477.6,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,48317.0,543.1,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5435.0,47.3,72093,Maricao,"{""72093"": 82.26, ""72153"": 11.67, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25413.0,264.4,72011,Añasco,"{""72011"": 96.8, ""72099"": 2.83, ""72083"": 0.37}",Añasco|Moca|Las Marías,72011|72099|72083,False,False,America/Puerto_Rico


In [147]:
# utils
def zipcode_rectifier(zip_code):
    assert type(zip_code) == int, f"Zip code must be an integer but got {type(zip_code)}"
    if zip_code < 100:
        zip_code = f'000{zip_code}'
    elif zip_code < 1000:
        zip_code = f'00{zip_code}'
    elif zip_code < 10000:
        zip_code = f'0{zip_code}'
    else:
        zip_code = f'{zip_code}'
    return zip_code

assert zipcode_rectifier(6000) == '06000'
assert zipcode_rectifier(12345) == '12345'
assert zipcode_rectifier(34) == '00034'

def list_zipcode_rectifier(zip_code_list):
    return list(map(zipcode_rectifier, zip_code_list))


In [148]:
out_folder = 'out/public'
os.makedirs(out_folder, exist_ok=True)

In [149]:

# Create a dictionary of city to zip code for each state
state_out_folder = f'{out_folder}/state'
os.makedirs(state_out_folder, exist_ok=True)


for state_id in df_zipcode['state_id'].unique():
    print('Processing state: ', state_id, '')
    os.makedirs(f'{state_out_folder}/{state_id}', exist_ok=True)
    state_df = df_zipcode.loc[df_zipcode['state_id'] == state_id]
    # state_df = df_zipcode.loc[df_zipcode['state_id'] == 'NC']
    # create every city on the state
    for city in state_df['city'].unique():
        city_df = state_df.loc[state_df['city'] == city].groupby('city').agg({'zip': list})
        city_df['zip'] = city_df['zip'].apply(lambda x: list_zipcode_rectifier(x))
        # city_df = state_df.loc[state_df['city'] == 'Durham'].groupby('city').agg({'zip': list})
        city_json = city_df.to_dict(orient='records')[0]
        city_json['city'] = city
        city_json['state'] = state_id
        city_row = state_df.iloc[0]
        city_json['lat'] = city_row.loc['lat']
        city_json['lng'] = city_row.lng
        city_json['state_name'] = city_row.state_name
        city_json['timezone'] = city_row.timezone
        with open(f'{state_out_folder}/{state_id}/{city}.json', 'w') as json_file:
            json.dump(city_json, json_file, indent=4)
            print(f"Created file: {state_out_folder}/{state_id}/{city}.json")
    state_df['zip'] = state_df.apply(lambda x: zipcode_rectifier(x['zip']), axis=1)
    city_zip_map = state_df.to_dict(orient='records')
    with open(f'{state_out_folder}/{state_id}.json', 'w') as json_file:
        json.dump(city_zip_map, json_file, indent=4)
        print(f"Created file: {state_out_folder}/{state_id}.json")


Processing state:  PR 
Created file: out/public/state/PR/Adjuntas.json
Created file: out/public/state/PR/Aguada.json
Created file: out/public/state/PR/Aguadilla.json
Created file: out/public/state/PR/Maricao.json
Created file: out/public/state/PR/Anasco.json
Created file: out/public/state/PR/Angeles.json
Created file: out/public/state/PR/Arecibo.json
Created file: out/public/state/PR/Bajadero.json
Created file: out/public/state/PR/Barceloneta.json
Created file: out/public/state/PR/Boqueron.json
Created file: out/public/state/PR/Cabo Rojo.json
Created file: out/public/state/PR/Penuelas.json
Created file: out/public/state/PR/Camuy.json
Created file: out/public/state/PR/Castaner.json
Created file: out/public/state/PR/Sabana Grande.json
Created file: out/public/state/PR/Ciales.json
Created file: out/public/state/PR/Utuado.json
Created file: out/public/state/PR/Dorado.json
Created file: out/public/state/PR/Ensenada.json
Created file: out/public/state/PR/Florida.json
Created file: out/public

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Created file: out/public/state/MA/Agawam.json
Created file: out/public/state/MA/Amherst.json
Created file: out/public/state/MA/Barre.json
Created file: out/public/state/MA/Belchertown.json
Created file: out/public/state/MA/Blandford.json
Created file: out/public/state/MA/Bondsville.json
Created file: out/public/state/MA/Brimfield.json
Created file: out/public/state/MA/Chester.json
Created file: out/public/state/MA/Chesterfield.json
Created file: out/public/state/MA/Chicopee.json
Created file: out/public/state/MA/Cummington.json
Created file: out/public/state/MA/Easthampton.json
Created file: out/public/state/MA/East Longmeadow.json
Created file: out/public/state/MA/East Otis.json
Created file: out/public/state/MA/Feeding Hills.json
Created file: out/public/state/MA/Gilbertville.json
Created file: out/public/state/MA/Goshen.json
Created file: out/public/state/MA/Granby.json
Created file: out/public/state/MA/Granville.json
Created file: out/public/state/MA/Hadley.json
Created file: out/p

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Created file: out/public/state/RI/Oakland.json
Created file: out/public/state/RI/Pascoag.json
Created file: out/public/state/RI/Pawtucket.json
Created file: out/public/state/RI/Central Falls.json
Created file: out/public/state/RI/Cumberland.json
Created file: out/public/state/RI/Lincoln.json
Created file: out/public/state/RI/Portsmouth.json
Created file: out/public/state/RI/Prudence Island.json
Created file: out/public/state/RI/Rockville.json
Created file: out/public/state/RI/Saunderstown.json
Created file: out/public/state/RI/Shannock.json
Created file: out/public/state/RI/Slatersville.json
Created file: out/public/state/RI/Tiverton.json
Created file: out/public/state/RI/Wakefield.json
Created file: out/public/state/RI/Kingston.json
Created file: out/public/state/RI/Narragansett.json
Created file: out/public/state/RI/Warren.json
Created file: out/public/state/RI/Warwick.json
Created file: out/public/state/RI/Westerly.json
Created file: out/public/state/RI/West Kingston.json
Created fi

...

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Created file: out/public/state/OR.json
Processing state:  WA 
Created file: out/public/state/WA/Auburn.json
Created file: out/public/state/WA/Federal Way.json
Created file: out/public/state/WA/Bellevue.json
Created file: out/public/state/WA/Black Diamond.json
Created file: out/public/state/WA/Bothell.json
Created file: out/public/state/WA/Carnation.json
Created file: out/public/state/WA/Duvall.json
Created file: out/public/state/WA/Edmonds.json
Created file: out/public/state/WA/Enumclaw.json
Created file: out/public/state/WA/Fall City.json
Created file: out/public/state/WA/Hobart.json
Created file: out/public/state/WA/Issaquah.json
Created file: out/public/state/WA/Kenmore.json
Created file: out/public/state/WA/Kent.json
Created file: out/public/state/WA/Kirkland.json
Created file: out/public/state/WA/Lynnwood.json
Created file: out/public/state/WA/Maple Valley.json
Created file: out/public/state/WA/Medina.json
Created file: out/public/state/WA/Mercer Island.json
Created file: out/publ

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Created file: out/public/state/AK/Holy Cross.json
Created file: out/public/state/AK/Homer.json
Created file: out/public/state/AK/Hooper Bay.json
Created file: out/public/state/AK/Hope.json
Created file: out/public/state/AK/Iliamna.json
Created file: out/public/state/AK/Kalskag.json
Created file: out/public/state/AK/Karluk.json
Created file: out/public/state/AK/Kasigluk.json
Created file: out/public/state/AK/Kasilof.json
Created file: out/public/state/AK/Kenai.json
Created file: out/public/state/AK/King Cove.json
Created file: out/public/state/AK/King Salmon.json
Created file: out/public/state/AK/Kipnuk.json
Created file: out/public/state/AK/Kodiak.json
Created file: out/public/state/AK/Kotlik.json
Created file: out/public/state/AK/Kwethluk.json
Created file: out/public/state/AK/Kwigillingok.json
Created file: out/public/state/AK/Wasilla.json
Created file: out/public/state/AK/Larsen Bay.json
Created file: out/public/state/AK/Levelock.json
Created file: out/public/state/AK/Lower Kalskag.

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [150]:
# create a per zip json object
df_zipcode = pd.read_csv(csv_path)
zip_out_folder = f'{out_folder}/zipcode'
os.makedirs(zip_out_folder, exist_ok=True)
for i in range(len(df_zipcode)):
    zip_code = zipcode_rectifier(int(df_zipcode.iloc[i].zip))
    file_name = f'{zip_out_folder}/{zip_code}.json'
    with open(file_name, 'w') as json_file:
        json.dump(df_zipcode.iloc[i].to_dict(), json_file, indent=4)
        print(f"Created file: {file_name}")


Created file: out/public/zipcode/00601.json
Created file: out/public/zipcode/00602.json
Created file: out/public/zipcode/00603.json
Created file: out/public/zipcode/00606.json
Created file: out/public/zipcode/00610.json
Created file: out/public/zipcode/00611.json
Created file: out/public/zipcode/00612.json
Created file: out/public/zipcode/00616.json
Created file: out/public/zipcode/00617.json
Created file: out/public/zipcode/00622.json
Created file: out/public/zipcode/00623.json
Created file: out/public/zipcode/00624.json
Created file: out/public/zipcode/00627.json
Created file: out/public/zipcode/00631.json
Created file: out/public/zipcode/00637.json
Created file: out/public/zipcode/00638.json
Created file: out/public/zipcode/00641.json
Created file: out/public/zipcode/00646.json
Created file: out/public/zipcode/00647.json
Created file: out/public/zipcode/00650.json
Created file: out/public/zipcode/00652.json
Created file: out/public/zipcode/00653.json
Created file: out/public/zipcode